# Load libraries

In [4]:
import pandas as pd
import numpy as np
import zipfile
import sqlite3
import matplotlib.pyplot as plt

# unzip compressed data

In [5]:
# path_to_zip = './data/ml-25m.zip'
# with zipfile.ZipFile(path_to_zip,"r") as zip_ref:
#     zip_ref.extractall("./data")

In [6]:
data_sources = {
    # movielens
    'genome_scores'    : './data/ml-25m/genome-scores.csv',
    'genome_tags'      : './data/ml-25m/genome-tags.csv',
    'links'            : './data/ml-25m/links.csv',
    'movies'           : './data/ml-25m/movies.csv',
    'ratings'          : './data/ml-25m/ratings.csv',
    'tags'             : './data/ml-25m/tags.csv',
#     imdb
    'imdb_name_basics' : './data/imdb/name.basics.tsv.gz',
    'imdb_title_basics': './data/imdb/title.basics.tsv.gz',
#     'imdb_ratings'     : './data/title.ratings.tsv.gz',
    'imdb_crew'        : './data/imdb/title.crew.tsv.gz',
    'imdb_principals'  : './data/imdb/title.principals.tsv.gz',
}

In [7]:
dfs = {}
for name, path in data_sources.items():
    print(dfs.keys(), end='\r')
    separator = '\t' if 'tsv' in path else ','
    dfs[name] = pd.read_csv(path, sep=separator).replace(to_replace=r'\N', value=np.nan)

/Users/ms621y/anaconda3/envs/DVA/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
dfs['genome_scores'].tag_id.nunique()

AttributeError: 'DataFrame' object has no attribute 'tag_id'

In [9]:
dfs['imdb_principals']

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0374658,cinematographer,director of photography,NaN
3,tt0000002,1,nm0721526,director,NaN,NaN
4,tt0000002,2,nm1335271,composer,NaN,NaN
...,...,...,...,...,...,...
38631696,tt9916880,5,nm0996406,director,principal director,NaN
38631697,tt9916880,6,nm1482639,writer,NaN,NaN
38631698,tt9916880,7,nm2586970,writer,books,NaN
38631699,tt9916880,8,nm1594058,producer,producer,NaN


In [10]:
dfs['genome_scores'].fk_id.nunique()

AttributeError: 'DataFrame' object has no attribute 'fk_id'

In [11]:
dfs['genome_scores']

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075
...,...,...,...
15584443,206499,1124,0.11000
15584444,206499,1125,0.04850
15584445,206499,1126,0.01325
15584446,206499,1127,0.14025


In [12]:
dfs['genome_scores'][dfs['genome_scores'].fk_id == 'tt0032544']

AttributeError: 'DataFrame' object has no attribute 'fk_id'

In [13]:
# crew is director/writer
# principal is cast (actors/resses)

In [14]:
# dfs['imdb_title_basics'].head()

In [15]:
# dfs['imdb_crew'].head()

In [16]:
# dfs['imdb_name_basics'][dfs['imdb_name_basics']['nconst']=='nm0005690']

In [17]:
# dfs['imdb_crew']['writers'].head()

In [18]:
# get all of the directors from all of the movies we have in our dataset
# for each director we need all of the movies that the director was in
# once we get this set of films we can calculate the tag relevence
# values for the director by some aggregation metric (mean




In [19]:
# EDA 
# combined_df = pd.merge(dfs['genome_scores'], dfs['genome_tags'], on='tagId')
# combined_df.tag.value_counts()

In [20]:
# EDA
# combined_df[combined_df.relevance == 1].tag.unique()
# all_ones = dfs['genome_scores'][dfs['genome_scores'].relevance==1.0]

In [21]:
# all_ones

In [22]:
# dfs['movies'][dfs['movies'].movieId==180]

In [23]:
# dfs['genome_tags'][dfs['genome_tags'].tagId ==1081]

In [24]:
# EDA
# for name in data_sources:
#     print(name)
#     print(dfs[name].shape)
#     print(dfs[name].head())
#     print()

In [25]:
# filter the movies we need from imdb based on movies present in tags dataset
merged_df = pd.merge(dfs['genome_scores'],
                     dfs['links'],
                     left_on='movieId',
                     right_on='movieId')
imdb_movies_to_keep = merged_df.imdbId.unique()

# properly format in order to join
imdb_ids_to_keep = 'tt' + pd.Series(imdb_movies_to_keep, dtype='str').str.zfill(7)

# filter imdb dataset
df =  dfs['imdb_title_basics']
dfs['imdb_title_basics_to_keep'] = df[df['tconst'].isin(imdb_ids_to_keep)]
dfs['imdb_title_basics_to_keep'].shape

(13795, 9)

In [26]:
dfs['genome_scores'].head()

,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [27]:
imdb_movies_to_keep

array([  114709,   113497,   113228, ...,  9243946, 10810424,  9398640])

In [28]:
dfs['imdb_title_basics_to_keep'].head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
11,tt0000012,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0,1896,NaN,1,"Action,Documentary,Short"
414,tt0000417,short,A Trip to the Moon,Le voyage dans la lune,0,1902,NaN,13,"Action,Adventure,Comedy"
436,tt0000439,short,The Great Train Robbery,The Great Train Robbery,0,1903,NaN,11,"Action,Crime,Short"
4912,tt0004972,movie,The Birth of a Nation,The Birth of a Nation,0,1915,NaN,195,"Drama,History,War"
6258,tt0006333,movie,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",0,1916,NaN,105,"Action,Adventure,Sci-Fi"


In [29]:
table_cols = {
    'tags'         : ['id', 'name'],
    'movies'       : ['id', 'kind', 'primary_title', 'original_title',
                      'release_year', 'runtime_minutes', 'genres'],
    'tag_relevance': ['movie_id', 'tag_id', 'relevance'],
}

In [30]:
# dfs['imdb_title_basics_to_keep'].apply(lambda x: int(x['tconst'][2:]), axis=1)

In [31]:
# dfs['imdb_title_basics_to_keep']['tconst'].copy().str[2:].copy().astype(int)

In [32]:
df_temp = dfs['imdb_title_basics_to_keep'].copy()
# dfs['imdb_title_basics_to_keep'] = df_temp

In [33]:
# set dfs with proper column names
keep_cols = ['id', 'titleType', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres']
# dfs['imdb_title_basics_to_keep'].loc[:, 'id'] = dfs['imdb_title_basics_to_keep'].apply(lambda x: int(x['tconst'][2:]), axis=1)
dfs['imdb_title_basics_to_keep'].loc[:,'id'] = dfs['imdb_title_basics_to_keep']['tconst'].copy()
# dfs['imdb_title_basics_to_keep'].loc[:,'id'] = dfs['imdb_title_basics_to_keep']['tconst'].str[2:].astype(int)  # to turn into int use this
dfs['imdb_title_basics_to_keep'] = dfs['imdb_title_basics_to_keep'][keep_cols]

# specify columns to keep
dfs['imdb_title_basics_to_keep'].columns = ['id', 'kind', 'primary_title', 'original_title',
                      'release_year', 'runtime_minutes', 'genres']
dfs['genome_tags'].columns = ['id', 'name']
dfs['genome_scores'].columns = ['fk_id', 'tag_id', 'relevance']
dfs['genome_scores']['fk_id'] = 'tt'+dfs['genome_scores']['fk_id'].astype(str).str.zfill(7)

/Users/ms621y/anaconda3/envs/DVA/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/ms621y/anaconda3/envs/DVA/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [34]:
15584448/1128

13816.0

In [35]:
tags_table = """
CREATE TABLE IF NOT EXISTS tags (
    id integer PRIMARY KEY,
    name text NOT NULL
);
"""

# data from the title.basics.tsv.gz
# TODO: format genre as json or extract out into own table
movies_table = """
CREATE TABLE IF NOT EXISTS movies (
    id text PRIMARY KEY NOT NULL,
    kind text,
    primary_title text,
    original_title text,
    release_year integer,
    runtime_minutes integer,
    genres text
);
"""

# movie_id is the imdb movie db id (not the movie lens movie id)
# to access this movie in a url add 'tt' to the beginning of it
# https://www.imdb.com/title/tt<movie_id>
# the fk_id can be any entity (actor, director, movie)
relevance_table = """
CREATE TABLE IF NOT EXISTS tag_relevance (
    fk_id text NOT NULL,
    tag_id name text NOT NULL,
    relevance real NOT NULL
);
"""
# 15584448 count in db


# TODO: create table containing columns each representing a tagname
# and rows each representing a movie, values being the relavance value
# this will make querying easier for similarity metric

#TODO check out themovieddb.org and see if there is any dataset that may 
# have some information we would want, if so we can join to this dataset as well.


In [36]:
db_name = './movie_sqlite.db'
conn = sqlite3.connect(db_name)
c = conn.cursor()

In [37]:
table_sql = {
    'tags'         : tags_table,
    'movies'       : movies_table,
    'tag_relevance': relevance_table
}

table_data_source = {
    'tags'         : 'genome_tags',
    'movies'       : 'imdb_title_basics_to_keep',
    'tag_relevance': 'genome_scores',
}

for table_name in table_sql:
    
    # create table
    c.execute(table_sql[table_name])
    
    # populate table with data
    df = dfs[table_data_source[table_name]]
    df.to_sql(table_name, conn, if_exists='append', index=False)

In [38]:
dfs['genome_scores'].head()

,fk_id,tag_id,relevance
0,tt0000001,1,0.02875
1,tt0000001,2,0.02375
2,tt0000001,3,0.06250
3,tt0000001,4,0.07575
4,tt0000001,5,0.14075


In [41]:
dfs['genome_scores'].head()

,fk_id,tag_id,relevance
0,tt0000001,1,0.02875
1,tt0000001,2,0.02375
2,tt0000001,3,0.06250
3,tt0000001,4,0.07575
4,tt0000001,5,0.14075


In [42]:
conn.close()

In [44]:
# sample sql to get all tag relevance scores for a movie
"""
    select m.primary_title, t.name, tr.relevance
    from movies m
    join tags t
    join tag_relevance tr
    on m.id = tr.movie_id
    and t.id = tr.tag_id
    where m.id = 12
    order by 3;
"""

'\n    select m.primary_title, t.name, tr.relevance\n    from movies m\n    join tags t\n    join tag_relevance tr\n    on m.id = tr.movie_id\n    and t.id = tr.tag_id\n    where m.id = 12\n    order by 3;\n'